In [1]:
import sympy as sym

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

In [3]:
from sympy.physics.mechanics import dynamicsymbols 
from sympy.physics.vector import *
from sympy.vector import CoordSys3D
N = CoordSys3D('N')

In [11]:
# joint configuration and velocity are functions of time
t, m1, m2, m3, Lc1, Lc2, Lc3, L1, L2, L3, I1, I2, I3, g = sym.symbols("t m1 m2 m3 Lc1 Lc2 Lc3 L1 L2 L3 I1 I2 I3 g")
q = sym.Array([sym.Function("θ1")(t), sym.Function("θ2")(t), sym.Function("θ3")(t)])
dq = q.diff(t)

# link poses
P1 = sym.Matrix([Lc1*sym.cos(q[0]),Lc1*sym.sin(q[0]), 
                 0])
P2 =sym.Matrix([L1*sym.cos(q[0]) + Lc2*sym.cos(q[0] + q[1]), L1*sym.sin(q[0]) + Lc2*sym.sin(q[0] + q[1]),
                0])
P3 = sym.Matrix([L1*sym.cos(q[0]) + L2*sym.cos(q[0] + q[1]) + Lc3 * sym.cos(q[0] + q[1] + q[2]), L1*sym.sin(q[0]) + L2*sym.sin(q[0] + q[1]) + Lc3 * sym.sin(q[0] + q[1] + q[2]), 0])
P3

Matrix([
[L1*cos(θ1) + L2*cos(θ1 + θ2) + Lc3*cos(θ1 + θ2 + θ3)],
[L1*sin(θ1) + L2*sin(θ1 + θ2) + Lc3*sin(θ1 + θ2 + θ3)],
[                                                   0]])

In [5]:

    # link Jacobians
J1 = P1.jacobian(q)
J2 = P2.jacobian(q)
J3 = P3.jacobian(q)
J2


Matrix([
[-L1*sin(θ1) - Lc2*sin(θ1 + θ2), -Lc2*sin(θ1 + θ2), 0],
[ L1*cos(θ1) + Lc2*cos(θ1 + θ2),  Lc2*cos(θ1 + θ2), 0],
[                             0,                 0, 0]])

In [12]:
import numpy as np


# inertia matrices
G1 = np.diag(np.array([m1, m1, m3]))
G2 = np.diag(np.array([m2, m2, m3]))
G3 = np.diag(np.array([m2, m2, m3]))

I = np.array([[I1+I2+I3,I2+I3,I3],[I2+I3, I2+I3, I3],[I3,I3,I3]])

# gravity
I

array([[I1 + I2 + I3, I2 + I3, I3],
       [I2 + I3, I2 + I3, I3],
       [I3, I3, I3]], dtype=object)

In [13]:
# mass matrix
M = (J1.transpose() * G1 * J1 + J2.transpose() * G2 * J2 + J3.transpose() * G3 * J3) + I

# Christoffel symbols and Coriolis matrix
dMdq = M.diff(q)
Γ = sym.permutedims(dMdq, (2, 1, 0)) - 0.5 * dMdq
C = sym.tensorcontraction(sym.tensorproduct(dq, Γ), (0, 2)).tomatrix()

# gravity vector
V = g * (m1 * P1[1] + m2 * P2[1] + m3 * P3[1])
g = V.diff(q)

# compile functions to numerical code
mass_matrix = sym.lambdify([q], M)
coriolis_matrix = sym.lambdify([q, dq], C)
gravity_vector = sym.lambdify([q], g)
def force(cls, q, dq, ddq):
        M = cls.mass_matrix(q)
        C = cls.coriolis_matrix(q, dq)
        g = cls.gravity_vector(q)

        return M @ ddq + C @ dq + g
M.simplify()

In [8]:
C.simplify()

In [15]:
M


Matrix([
[I1 + I2 + I3 + 2*L1**2*m2 + 2*L1*L2*m2*cos(θ2) + 2*L1*Lc2*m2*cos(θ2) + 2*L1*Lc3*m2*cos(θ2 + θ3) + L2**2*m2 + 2*L2*Lc3*m2*cos(θ3) + Lc1**2*m1 + Lc2**2*m2 + Lc3**2*m2, I2 + I3 + L1*L2*m2*cos(θ2) + L1*Lc2*m2*cos(θ2) + L1*Lc3*m2*cos(θ2 + θ3) + L2**2*m2 + 2*L2*Lc3*m2*cos(θ3) + Lc2**2*m2 + Lc3**2*m2, I3 + L1*Lc3*m2*cos(θ2 + θ3) + L2*Lc3*m2*cos(θ3) + Lc3**2*m2],
[                                    I2 + I3 + L1*L2*m2*cos(θ2) + L1*Lc2*m2*cos(θ2) + L1*Lc3*m2*cos(θ2 + θ3) + L2**2*m2 + 2*L2*Lc3*m2*cos(θ3) + Lc2**2*m2 + Lc3**2*m2,                                                                 I2 + I3 + L2**2*m2 + 2*L2*Lc3*m2*cos(θ3) + Lc2**2*m2 + Lc3**2*m2,                          I3 + L2*Lc3*m2*cos(θ3) + Lc3**2*m2],
[                                                                                                         I3 + L1*Lc3*m2*cos(θ2 + θ3) + L2*Lc3*m2*cos(θ3) + Lc3**2*m2,                                                                                               I3 + L2*Lc3*